In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

## Implementing the Logical Expression Oracle
Logical expressions are commonly used to describe problems, particularly those that have some constraints. These logical expressions can be used to construct a circuit and execute it on various algorithms. Let's begin with a simple problem.

Imagine you need to organize your Saturday conference call with your friends: Marco, Carlo, Robert and Lara.

Here is the problem you have been asked to solve:
- in the last period Marco Carlo do not get along very well;
- Lara and Robert become very close and they are fine together;
- because of that, Marco and Robert broke up.

What you need to do is determine which combination of your friends is best for you to for the call, and let it goes with minimal issues based on their history together.

To solve this, let's write this out as a logical expression:
1. We'll map each friend as A = Marco, B = Carlo, C = Robert, and D = Lara.
2. Next, we'll create a logical expression using logical operators to illustrate the constraints. To start, we know that Marco and Carlo do not get along, so this we can represent as follows,
where ^ indicates XOR:
#### (A^B)
3. Next, we know that Robert and Lara get along fine together, so we can represent them as follows:
#### (C&D)
4. Finally, we know that Marco and Robert might not be open to see each other, so we will represent them as follows:
#### ~(A&C)
5. By putting these all together, our complete logical expression for this example is as follows:
### (A^B)&(C&D)&~(A&C)


Now that we have defined our logical expression, let's create an Oracle on the logical expression so that we can use **Grover's algorithm** to search for the optimal result

In [ ]:
# Import the necessary modules and classes
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import LogicalExpressionOracle, TruthTableOracle, CustomCircuitOracle

In [ ]:
# State the SAT problem into a logical expression
expression = '((Marco ^ Carlo) & (Robert & Lara) & ~(Marco & Robert))'

In [ ]:
 # Create an Oracle based on the Logical Expression
oracle = LogicalExpressionOracle(expression)

In [ ]:
# Construct the circuit from the oracle
quantum_circuit = oracle.construct_circuit()
quantum_circuit.draw('mpl')

In [ ]:
# Generate a quantum instance from a simulator
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'), shots=1024)

In [ ]:
# Create the Grover algorithm with the Logical Expression
# Oracle
grover = Grover(oracle)

In [ ]:
# Run the Grover algorithm
result = grover.run(quantum_instance)
# Print the top measured result
print('Top result:', result['top_measurement']) # Plot all measured results plot_histogram(result['measurement'])

The preceding code results in the following output. Keep in mind that the qubit at position 0 is represented by the least significant bit (far right).

This means that the result, 1011, is equal to Lara=1, Robert=0, Carlo=1, Marco=1

In [ ]:
plot_histogram(result.measurement, title='Call with Friends', bar_labels = True)

## Experiment with Real Device

In [ ]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= quantum_circuit.num_qubits and 
                                   not x.configuration().simulator and x.status().operational==True))
print("least busy backend: ", backend)


In [ ]:
from qiskit.compiler import transpile

# transpile the circuit for ibmq_16_melbourne
grover_compiled = transpile(result['circuit'], backend=backend, optimization_level=3)

print('gates = ', grover_compiled.count_ops())
print('depth = ', grover_compiled.depth())


In [ ]:
grover_compiled.draw()

In [ ]:
# Generate a quantum instance for the HW
quantum_instance_hw = QuantumInstance(backend, shots=1024, optimization_level=3)



The number of gates needed is far above the limits regarding decoherence time of the current near-term quantum computers. It is a challenge to design a quantum circuit for Grover search to solve satisfiability and other optimization problems.

In [ ]:
%qiskit_job_watcher

In [ ]:
from qiskit.tools.monitor import job_monitor
job = grover.run(quantum_instance_hw)
#job = execute(grover_circuit, backend=backend, shots=1024, optimization_level=3)
job_monitor(job, interval = 2)